# PyFtdi RPi.GPIO test
http://eblot.github.io/pyftdi/  
https://pypi.org/project/pyftdi/  
https://github.com/eblot/pyftdi  
https://www.ftdichip.com/  
https://www.ftdichip.com/Products/ICs/FT232H.htm  

Pinout:  
http://eblot.github.io/pyftdi/pinout.html  

OLED display:  
https://circuitdigest.com/article/ssd1306-oled-display  

### 需安裝

```
pip install pyusb

```

### Windows DLL
[libusb-win32-devel-filter-1.2.6.0.exe](https://sourceforge.net/projects/libusb-win32/files/libusb-win32-releases/1.2.6.0/)  

https://sourceforge.net/projects/libusb-win32/  



### Linux 權限
https://ubuntuforums.org/showthread.php?t=1682084  
https://linuxconfig.org/tutorial-on-how-to-write-basic-udev-rules-in-linux  
```
If anyone is interested.. fixed with udev rule:

cd /etc/udev/rules.d/

sudo nano 70-usb.rules

SUBSYSTEM=="usb", ATTRS{idVendor}=="1a86", GROUP="dialout", MODE="666" 
SUBSYSTEM=="usb", ATTRS{idVendor}=="0403", GROUP="dialout", MODE="666" 

then reloaded udev rules:

sudo udevadm control --reload-rules  
sudo udevadm trigger

```

### 安裝步驟

- 安裝 [libusb-win32-devel-filter](https://sourceforge.net/projects/libusb-win32/)
    - [libusb-win32-devel-filter-1.2.6.0.exe](https://sourceforge.net/projects/libusb-win32/files/libusb-win32-releases/1.2.6.0/) 
- 使用 libUSB-Win32, install all class filters    


- 安裝 PySerial
```
pip install pyserial
```
    
- 安裝 PyUSB
```
pip install pyusb
``` 

- 安裝 PyFtdi (http://eblot.github.io/pyftdi/installation.html)
```
pip install pyftdi
``` 

In [2]:
%pylab inline  

Populating the interactive namespace from numpy and matplotlib


In [3]:
import os
import sys

sys.path.append(os.path.abspath(os.path.join('..', 'codes')))

path = os.sep.join(['..', '..', '..', '..', '..', '待處理', 'ORM', 'bitbucket', 'github', 'codes'])
sys.path.append(path) 

path = os.sep.join(['..', '..', '..', '..', '..', '待處理', 'USB', 'Universal Serial Bus', 'bitbucket', 'github', 'codes'])
sys.path.append(path)

import universal_serial_bus
from universal_serial_bus import *

import pyftdi
from pprint import pprint

## RPi.GPIO
https://sourceforge.net/p/raspberry-gpio-python/wiki/Examples/  

## BasicUsage
### Importing the module

In [4]:
# import RPi.GPIO as GPIO
from bridges.ftdi.adapters.rpi.RPi import GPIO

### Pin numbering
The mode will be GPIO.BOARD, GPIO.BCM or None

In [5]:
GPIO.setmode(GPIO.BOARD) 
GPIO.setmode(GPIO.BCM)

mode = GPIO.getmode()
mode

11

### Warnings

In [6]:
# GPIO.setwarnings(False)

### Setup up a channel

In [7]:
channel = 6

GPIO.setup(channel, GPIO.IN)

GPIO.setup(channel, GPIO.OUT)

GPIO.setup(channel, GPIO.OUT, initial=GPIO.HIGH)

### Setup more than one channel

In [8]:
chan_list = [5, 6]    # add as many channels as you want!
                       # you can tuples instead i.e.:
                       #   chan_list = (11,12)
GPIO.setup(chan_list, GPIO.OUT)

### Input
This will return either 0 / GPIO.LOW / False or 1 / GPIO.HIGH / True.

In [9]:
GPIO.setup(channel, GPIO.IN)
GPIO.input(channel)

1

### Output
tate can be 0 / GPIO.LOW / False or 1 / GPIO.HIGH / True.

In [10]:
channel = 6
state = GPIO.HIGH

GPIO.setup(channel, GPIO.OUT)
GPIO.output(channel, state)

### Output to several channels

In [11]:
chan_list = [5, 6]                             # also works with tuples
GPIO.setup(chan_list, GPIO.OUT)
GPIO.output(chan_list, GPIO.LOW)                # sets all to GPIO.LOW
GPIO.output(chan_list, (GPIO.HIGH, GPIO.LOW))   # sets first HIGH and second LOW

### Cleanup

In [12]:
# GPIO.cleanup()

In [13]:
# channel1 = 5
# channel2 = 6

# GPIO.cleanup(channel)
# GPIO.cleanup( (channel1, channel2) )
# GPIO.cleanup( [channel1, channel2] )

### RPi Board Information and RPi.GPIO version

In [14]:
GPIO.RPI_INFO 

{'PLATFORM': 'Windows-10-10.0.17763-SP0',
 'P1_REVISION': 'Windows-10-10.0.17763-SP0'}

In [15]:
GPIO.RPI_INFO['P1_REVISION'] 

'Windows-10-10.0.17763-SP0'

In [16]:
GPIO.VERSION

0.1

## Inputs

### Pull up / Pull down resistors
not sure FTDI ICs support this ?

In [17]:
from bridges.ftdi.adapters.rpi.RPi import GPIO

channel = 6
GPIO.setup(channel, GPIO.OUT, initial = GPIO.HIGH)

In [18]:
channel = 5

GPIO.setup(channel, GPIO.IN, pull_up_down=GPIO.PUD_UP)

### Testing inputs (polling)

In [19]:
if GPIO.input(channel):
    print('Input was HIGH')
else:
    print('Input was LOW')

Input was HIGH


In [20]:
# while GPIO.input(channel) == GPIO.HIGH:
#     time.sleep(0.01)  # wait 10 ms to give CPU chance to do other things
    
# print('It is LOW now!')

### Interrupts and Edge detection (not supported, FTDI IC doesn't have USB endpoint of "Interrupt input" type.)
#### wait_for_edge() function
 
Note that you can detect edges of type GPIO.RISING, GPIO.FALLING or GPIO.BOTH. The advantage of doing it this way is that it uses a negligible amount of CPU, so there is plenty left for other tasks.

In [21]:
# GPIO.wait_for_edge(channel, GPIO.RISING)

In [22]:
# wait for up to 5 seconds for a rising edge (timeout is in milliseconds)
# channel = GPIO.wait_for_edge(channel, GPIO_RISING, timeout=5000)

if channel is None:
    print('Timeout occurred')
else:
    print('Edge detected on channel', channel)

Edge detected on channel 5


#### event_detected() function

In [23]:
# GPIO.add_event_detect(channel, GPIO.RISING)  # add rising edge detection on a channel

# if GPIO.event_detected(channel):
#     print('Button pressed')

### Threaded callbacks
RPi.GPIO runs a second thread for callback functions. This means that callback functions can be run at the same time as your main program, in immediate response to an edge. For example:

In [24]:
# def my_callback(channel):
#     print('This is a edge event callback function!')
#     print('Edge detected on channel %s'%channel)
#     print('This is run in a different thread to your main program')
    
# """
# GPIO.add_event_detect(channel, GPIO.RISING, callback=my_callback)  # add rising edge detection on a channel
# ...the rest of your program...
# If you wanted more than one callback function:
# """

# def my_callback_one(channel):
#     print('Callback one')

# def my_callback_two(channel):
#     print('Callback two')

### Switch debounce

In [25]:
# GPIO.add_event_detect(channel, GPIO.RISING, callback=my_callback, bouncetime=200) 

# GPIO.add_event_callback(channel, my_callback, bouncetime=200)

### Remove event detection

In [26]:
# GPIO.remove_event_detect(channel)

## Outputs

In [27]:
"1. First set up RPi.GPIO (as described here)"

# import RPi.GPIO as GPIO
from bridges.ftdi.adapters.rpi.RPi import GPIO 

GPIO.setmode(GPIO.BOARD)

channel = 6
GPIO.setup(channel, GPIO.OUT)

In [28]:
"2. To set an output high:"

GPIO.output(channel, GPIO.HIGH)
" # or"
GPIO.output(channel, 1)
" # or"
GPIO.output(channel, True)

In [29]:
"3. To set an output low:"
GPIO.output(channel, GPIO.LOW)
" # or"
GPIO.output(channel, 0)
" # or"
GPIO.output(channel, False)

In [30]:
"4. To output to several channels at the same time:"
chan_list = (channel, 5)
GPIO.output(chan_list, GPIO.LOW) # all LOW
GPIO.output(chan_list, (GPIO.HIGH,GPIO.LOW))  # first LOW, second HIGH

In [31]:
"Note that you can read the current state of a channel set up as an output using the input() function. For example to toggle an output:"
GPIO.output(channel, not GPIO.input(channel))

In [32]:
"5. Clean up at the end of your program"
# GPIO.cleanup()

'5. Clean up at the end of your program'

## PWM (Not Implemented)

In [33]:
# import RPi.GPIO as GPIO
from bridges.ftdi.adapters.rpi.RPi import GPIO 
 
GPIO.setmode(GPIO.BOARD)

channel = 6
GPIO.setup(channel, GPIO.OUT)

In [34]:
# # "To create a PWM instance:"
# frequency = 50
# p = GPIO.PWM(channel, frequency)

In [35]:
# # "To start PWM:"
# dc = 50

# p.start(dc)   # where dc is the duty cycle (0.0 <= dc <= 100.0)

In [36]:
# # "To change the frequency:"
# freq = 50
# p.ChangeFrequency(freq)   # where freq is the new frequency in Hz

In [37]:
# # "To change the duty cycle:"

# p.ChangeDutyCycle(dc)  # where 0.0 <= dc <= 100.0

In [38]:
# # "To stop PWM:"

# p.stop()

In [39]:
# # "Note that PWM will also stop if the instance variable 'p' goes out of scope."

# # "An example to blink an LED once every two seconds:"
 
# GPIO.setmode(GPIO.BOARD)
# GPIO.setup(channel, GPIO.OUT)

# p = GPIO.PWM(channel, 0.5)
# p.start(1)

# input('Press return to stop:')   # use raw_input for Python 2
# p.stop()
# GPIO.cleanup()

In [40]:
# # "An example to brighten/dim an LED:"

# import time 

# GPIO.setmode(GPIO.BOARD)
# GPIO.setup(channel, GPIO.OUT)

# p = GPIO.PWM(channel, 50)  # channel=channel frequency=50Hz
# p.start(0)

# try:
    
#     while 1:
        
#         for dc in range(0, 101, 5):
#             p.ChangeDutyCycle(dc)
#             time.sleep(0.1)
            
#         for dc in range(100, -1, -5):
#             p.ChangeDutyCycle(dc)
#             time.sleep(0.1)
            
# except KeyboardInterrupt:
#     pass

# p.stop()
# GPIO.cleanup()

In [41]:
# GPIO.cleanup()

## gpio_function(channel)
will return a value from:  
GPIO.IN, GPIO.OUT, GPIO.SPI, GPIO.I2C, GPIO.HARD_PWM, GPIO.SERIAL, GPIO.UNKNOWN

In [42]:
# import RPi.GPIO as GPIO
from bridges.ftdi.adapters.rpi.RPi import GPIO 

GPIO.setmode(GPIO.BOARD)

channel = 6

GPIO.setup(channel, GPIO.IN)
print(GPIO.gpio_function(channel))

GPIO.setup(channel, GPIO.OUT)
print(GPIO.gpio_function(channel)) 

['IN']
['OUT']


### Clean up at the end of your program"

In [43]:
# Clean up at the end of your program"
GPIO.cleanup()